<a href="https://colab.research.google.com/github/pei0217/fin_hw6_week10/blob/main/fin_hw6_week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset


# 加载 Financial PhraseBank 数据集的 "sentences_allagree" 配置
dataset = load_dataset("takala/financial_phrasebank", "sentences_allagree", split="train")

# 数据预处理
texts = [sample['sentence'] for sample in dataset]
labels = [sample['label'] for sample in dataset]  # 标签: 0 = Negative, 1 = Neutral, 2 = Positive

from transformers import BertTokenizer

# 初始化 BERT 的分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 将文本编码为 BERT 输入格式
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

import torch

class FinancialDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

dataset = FinancialDataset(encodings, labels)

from transformers import BertForSequenceClassification

# 初始化预训练的 BERT 模型 (3 分类任务)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

from torch.utils.data import DataLoader
from transformers import AdamW

# 数据加载器
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

# 确保模型和数据在 GPU 上运行
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 训练模型
model.train()
for epoch in range(3):  # 3 个训练轮次
    for batch in dataloader:
        # 将数据移动到 GPU
        batch = {key: val.to(device) for key, val in batch.items()}

        # 前向传播
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

model.eval()
test_texts = [
    "The company's profit has increased significantly this quarter.",
    "The increase in costs negatively affected the revenue.",
    "The company's performance remained stable."
]
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)
outputs = model(**test_encodings)
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# 映射结果到标签
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
predicted_labels = [label_map[pred] for pred in preds]
print(predicted_labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for takala/financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/takala/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-3-222fc84cfaa2>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1 completed with loss: 0.013371452689170837
Epoch 2 completed with loss: 0.07115218788385391
Epoch 3 completed with loss: 0.0007682127179577947
['Positive', 'Negative', 'Positive']
